# Python und MySQL - Cursors DEMO

Das grösste Problem bei der Koppelung einer (3. Generations-) Programmiersprache wie z.B. C, Python, Cobol, Java, PHP, etc. mit einer (4. Generations-) Programmiersprache wie z.B. SQL, sind die unterschiedlichen Datenstrukturkonzepte der beiden Programmiersprachen-Ansätze. 

Imperative (3. Generations-) Programmiersprachen basieren auf der Datenstruktur Tupel (Datensatz) als Basiskonstrukt, während deklarative (4. Generations-) Programmiersprachen auf dem Konzept der Relation, also einer Menge von Tupeln als eine Einheit (eine Tabelle), beruht. 

#### Imperative Programmiersprachen sind "Satzorientiert", deklarative Programmiersprachen sind "Mengenorientiert". 

Dieser Gegensatz wird als <b>Impedance Mismatch</b> bezeichnet. Der Impedance Mismatch wird durch das Konstrukt des Cursors gelöst. Ein <b>Cursor</b> erlaubt, eine <b>Relation</b> (eine Menge von Tupeln - nämlich das Result-Set einer SQL-Abfrage), Zeile für Zeile (resp. eben Tupel für Tupel) abzuarbeiten.

<img src="MySQL_Python_Cursor.png">

In den folgenden Beispielen wird dies anhand verschiedener Varianten von Cursorn am konkreten Beispiel von Python mit MySQL demonstriert. Das Verfahren (das Konzept) ist bei allen imperativen Programmiersprachen gleich. Lediglich die einzelnen Bibliotheken (Libraries) und/oder Schlüsselworte sind sprachabhängig. Die einzelnen Beispiele sind jeweils komplett und können jedes einzeln für sich verwendet werden.

## Beispiel 1: Standard-Cursor

Der Standard-Cursor (Default-Cursor) der MySQLdb-Library in Python gibt eine Tabelle mit Spalten (an array of values) in der Reihenfolge der Abfrage (in the order you asked/retrieve them) zurück. 

<b>Was zum Teufel heisst das den?</b> 

Schauen wir uns das Schritt für Schritt an!

Um aus Python auf eine SQL-Datenbank zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. In unseren Beispielen arbeiten wir mit Python2 und MySQL.

In [ ]:
import mysql.connector


<b>Hinweis: </b>In den Beispielen wird davon ausgegangen, dass die uns wohlbekannte Datenbank "Heizungsmonteur" vorhanden ist. Ebenfalls ist ein MySQL-User mit Namen pkmlp und Password pkmlp vorhanden. Im pkmlpLernportal (www.pkmlp.ch) sind SQL-Scripts für das Erstellen der DB vorhanden (sofern diese DB nicht bereits schon seit langem installiert ist). In der pkmlpLernumgebung kann auch der installierte User root mit Password pkmlp verwendet werden. 

Als nächstes muss eine Verbindung zur Datenbank (mit gültigen Credentials - siehe Hinweis oben) hergestellt werden. Kann die Verbindung nicht aufgebaut werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab. 

In [ ]:
try:
    dbVerbindung = mysql.connector.connect(host = "localhost", 
                                   user = "pkmlp", 
                                   passwd = "pkmlp", 
                                   db = "Heizungsmonteur")

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print "Programm wird abgebrochen"
    # sys.exit(1)

else:
    print("Verbindung zu MySQL DB aufgebaut")


<b>Hinweis: </b>Hier im IPython-Notebook sind die Befehle zum Programmabbruch jeweils deaktiviert (auskommentiert). Wir sehen ja, wenn etwas schief läuft und können sofort reagieren/korrigieren. In Batch-Programmen ist es angebracht, im Fehlerfall das Programm mit der Ausgabe einer entsprechenden Fehlermeldung abzubrechen. In Dialog-Programmen ist es angebracht, den Benutzer darauf hinzuweisen und allenfalls eine erneute Eingabe zu verlangen. Dies sind aber Themen die ganze Programmierungs-Workshops füllen. Darum verzichten wir hier auf dies, damit wir uns wirklich auf die Bearbeitung von relationalen Datenbanken in einer imperativen Programmiersprache mit embedded SQL konzentrieren können.   

Konnte die Verbindung erfolgreich aufgebaut werden, so wird der Cursor definiert, über den die Abfragen an die Datenbank ausgeführt werden. Kann der Cursor nicht definiert werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor = dbVerbindung.cursor()

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print "Programm wird abgebrochen"
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("Cursor zu MySQL DB definiert")


Konnte der Cursor erfolgreich definiert werden, wird das SQL-Statement ausgeführt. Kann das SQL-Statement nicht erfolgreich ausgeführt werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor.execute("""SELECT M_Name, Abteilung, Funktion 
                          FROM Monteur, Abteilung, Funktion 
                         WHERE Monteur.A_Nr = Abteilung.A_Nr 
                           AND Monteur.F_Nr = Funktion.F_Nr;""")

except mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbCursor.close()
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("SQL-Statement ausgeführt")


Konnte das SQL-Statement erfolgreich ausgeführt werden, können nun die Zeilen (Row, Tuple, Datensatz) der Tabelle (resp. des Cursors, des Result-Sets) in einer Schlaufe ausgegeben werden. Dafür gibt es zwei verschiedene Varianten:


#### Variante 1: mit einem "normalem" Python-Iterator 

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter)


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter)
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

In beiden Varianten gibt der Cursor jede Zeile (jede Row, jeden Tupel, jeden Datensatz) als Python-Tupel zurück (Du erinnerst dich: Der Standard-Cursor (Default-Cursor) der MySQLdb-Library in Python gibt eine Tabelle mit Spalten (an array of values) in der Reihenfolge der Abfrage (in the order you asked/retrieve them) zurück). Was das heisst, hast Du nun gesehen. 

Um die einzelnen Werte (Attribute) ansprechen zu können, müssen wir diese über deren Index adressieren.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter[0], mitarbeiter[1], mitarbeiter[2])


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter[0], mitarbeiter[1], mitarbeiter[2])
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

Das Ansprechen der Spalten (Attribute) über deren Index ist nicht sehr sprechend. Sowohl in SQL, wie auch in Python (Python-Dictionary) sind wir es gewohnt, Attribute über den Attribute-Namen anzusprechen. Das zweite Beispiel wird zeigen, wie dies gemacht werden kann. Zuerst schliessen wir aber dieses Beispiel sauber ab.

Konnte alles erfolgreich durchgeführt werden, so muss wieder sauber aufgeräumt werden. Zuerst schliessen wir den Cursor wieder. 

In [ ]:
dbCursor.close()


Zum Abschluss schliessen wor die Verbindung zur Datenbank.

In [ ]:
dbVerbindung.close()


Damit haben wir Beispiel 1 abgeschlossen. 

## Beispiel 2: Dict-Cursor

 Der Dict-Cursor der MySQLdb-Library in Python gibt die Tabelle (das Result-Set) als Python-Dictionary zurück.
 
<b>Was zum Teufel heisst das den?</b> 

Schauen wir uns auch das Schritt für Schritt an!

Um aus Python auf eine SQL-Datenbank zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. In diesem Fall wolllen wir aber nicht mit dem Default-Cursor arbeiten, darum müssen wir zusätzlich noch die Cursor-Library einbinden.

In [ ]:
import mysql.connector


Als nächstes muss wieder eine Verbindung zur Datenbank (mit gültigen Credentials - siehe Hinweis oben in Beispiel 1) hergestellt werden. Zusätzlich definieren wir hier auch den Cursor (cursorclass) der verwendet werden soll. Kann die Verbindung nicht aufgebaut werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbVerbindung = mysql.connector.connect(host = "localhost", 
                                           user = "pkmlp", 
                                           passwd = "pkmlp", 
                                           db = "Heizungsmonteur")

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # sys.exit(1)

else:
    print("Verbindung zu MySQL DB aufgebaut")


Konnte die Verbindung erfolgreich aufgebaut werden, so wird der Cursor definiert, über den die Abfragen an die Datenbank ausgeführt werden. Kann der Cursor nicht definiert werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor = dbVerbindung.cursor(dictionary=True)

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("Cursor zu MySQL DB definiert")


Konnte der Cursor erfolgreich definiert werden, wird das SQL-Statement ausgeführt. Kann das SQL-Statement nicht erfolgreich ausgeführt werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor.execute("""SELECT M_Name, Abteilung, Funktion 
                          FROM Monteur, Abteilung, Funktion 
                         WHERE Monteur.A_Nr = Abteilung.A_Nr 
                           AND Monteur.F_Nr = Funktion.F_Nr;""")

except mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbCursor.close()
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("SQL-Statement ausgeführt")


Konnte das SQL-Statement erfolgreich ausgeführt werden, können nun die Zeilen (Tuple, Row, Datensatz) der Tabelle (resp. des Cursors, des Result-Sets) in einer Schlaufe ausgegeben werden. Auch hier haben wir wieder zwei Varianten zur Auswahl wie im ersten Beispiel:

#### Variante 1: mit einem "normalem" Python-Iterator 

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter)


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter)
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

In beiden Varianten gibt der Cursor jede Zeile (jede Row, jeden Tupel, jeden Datensatz) als Python-Dictionary zurück (Du erinnerst Dich: Der Dict-Cursor der MySQLdb-Library in Python gibt die Tabelle (das Result-Set) als Python-Dictionary zurück). Was das heisst, hast Du nun gesehen. 

Nun können wir aber die Attribute über deren Key (Namen) ansprechen.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter["M_Name"], mitarbeiter["Abteilung"], mitarbeiter["Funktion"])


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter["M_Name"], mitarbeiter["Abteilung"], mitarbeiter["Funktion"])
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

Das Ansprechen der Spalten (Attribute) über deren Namen ist deutlich sprechender.

Konnte alles erfolgreich durchgeführt werden, so muss wieder sauber aufgeräumt werden. Zuerst schliessen wir den Cursor wieder. 

In [ ]:
dbCursor.close()

Zum Abschluss schliessen wir auch noch die Verbindung zur Datenbank.

In [ ]:
dbVerbindung.close()

Damit haben wir auch Beispiel 2 erfolgreich abgeschlossen.

## Beispiel 3: Server Side Cursor

<b>Beachte:</b> Das Result-Set einer SQL-Abfrage wird als Ganzes vom Server an den Client übertragen. Auf der Client-Seite wird dann das Result-Set mit dem Cursor Row für Row (resp. Zeile für Zeile oder eben Datensatz für Datensatz) abgearbeitet. Dies stellt kein Problem dar, wenn das Result-Set nicht allzu gross ist. Bei sehr grossen Result-Sets kann dies zu zweierlei Problemen führen: 1. die Übertragung des Result-Sets dauert sehr (evtl. zu) lange, oder 2. das Result-Set ist zu gross für den (Speicher-) Platz auf der Client-Seite.

Sehr grosse Result-Sets sollten auf dem Server bleiben und nur die Row (der Tupel, der Datensatz) übertragen werden, der gerade verarbeitet wird. Dazu gibt es die Server-Seitigen Cursor: SSCursor resp. SSDictCursor. Und auch hier: Schritt für Schritt.

Um aus Python auf eine SQL-Datenbank zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. In diesem Fall wolllen wir aber nicht mit dem Default-Cursor arbeiten, darum müssen wir zusätzlich noch die Cursor-Library einbinden.

In [ ]:
import mysql.connector


Als nächstes muss wieder eine Verbindung zur Datenbank (mit gültigen Credentials - siehe Hinweis oben in Beispiel 1) hergestellt werden. Zusätzlich definieren wir hier auch den Cursor (cursorclass) der verwendet werden soll. Kann die Verbindung nicht aufgebaut werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbVerbindung = mysql.connector.connect(host = "localhost", 
                                           user = "pkmlp", 
                                           passwd = "pkmlp", 
                                           db = "Heizungsmonteur")

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # sys.exit(1)

else:
    print("Verbindung zu MySQL DB aufgebaut")


Konnte die Verbindung erfolgreich aufgebaut werden, so wird der Cursor definiert, über den die Abfragen an die Datenbank ausgeführt werden. Kann der Cursor nicht definiert werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor = dbVerbindung.cursor(buffered=True)

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("Cursor zu MySQL DB definiert")


Konnte der Cursor erfolgreich definiert werden, wird das SQL-Statement ausgeführt. Kann das SQL-Statement nicht erfolgreich ausgeführt werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor.execute("""SELECT M_Name, Abteilung, Funktion 
                          FROM Monteur, Abteilung, Funktion 
                         WHERE Monteur.A_Nr = Abteilung.A_Nr 
                           AND Monteur.F_Nr = Funktion.F_Nr;""")

except mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbCursor.close()
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("SQL-Statement ausgeführt")


Konnte das SQL-Statement erfolgreich ausgeführt werden, können nun die Zeilen (Tuple, Row, Datensatz) der Tabelle (resp. des Cursors, des Result-Sets) in einer Schlaufe ausgegeben werden. Auch hier haben wir wieder zwei Varianten.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter)


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter)
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

Wie die Beispiele zeigen, gibt der Cursor jede Zeile (jede Row, jeden Tupel, jeden Datensatz) als Python-Tuple zurück. Um die einzelnen Werte ansprechen zu können, müssen wir diese über deren Index ansprechen.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter[0], mitarbeiter[1], mitarbeiter[2])


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter[0], mitarbeiter[1], mitarbeiter[2])
    mitarbeiter = dbCursor.fetchone()
    

<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

Konnte alles erfolgreich durchgeführt werden, so muss wieder sauber aufgeräumt werden. Zuerst schliessen wir den Cursor wieder.

In [ ]:
dbCursor.close()


Zum Abschluss schliessen wir auch noch die Verbindung zur Datenbank.

In [ ]:
dbVerbindung.close()


Damit haben wir Beispiel 3 ebenfalls erfolgreich abgeschlossen.

## Beispiel 4: Server Side DictCursor

Natürlich kann auch auf dem Server ein DictCursor verwendet werden. Doch auch hier wieder: Schritt für Schritt.

Um aus Python auf eine SQL-Datenbank zugreifen zu können, muss die entsprechende Bibliothek eingebunden werden. In diesem Fall wolllen wir aber nicht mit dem Default-Cursor arbeiten, darum müssen wir zusätzlich noch die Cursor-Library einbinden.

In [ ]:
import mysql.connector


Als nächstes muss wieder eine Verbindung zur Datenbank (mit gültigen Credentials - siehe Hinweis oben in Beispiel 1) hergestellt werden. Zusätzlich definieren wir hier auch den Cursor (cursorclass) der verwendet werden soll. Kann die Verbindung nicht aufgebaut werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbVerbindung = mysql.connector.connect(host = "localhost", 
                                           user = "pkmlp", 
                                           passwd = "pkmlp", 
                                           db = "Heizungsmonteur")

except  mysql.connector.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # sys.exit(1)

else:
    print("Verbindung zu MySQL DB aufgebaut")


Konnte die Verbindung erfolgreich aufgebaut werden, so wird der Cursor definiert, über den die Abfragen an die Datenbank ausgeführt werden. Kann der Cursor nicht definiert werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor = dbVerbindung.cursor(dictionary=True, buffered=True)

except  MySQLdb.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("Cursor zu MySQL DB definiert")


Konnte der Cursor erfolgreich definiert werden, wird das SQL-Statement ausgeführt. Kann das SQL-Statement nicht erfolgreich ausgeführt werden, bricht das Programm mit einer entsprechenden Fehlermeldung ab.

In [ ]:
try:
    dbCursor.execute("""SELECT M_Name, Abteilung, Funktion 
                          FROM Monteur, Abteilung, Funktion 
                         WHERE Monteur.A_Nr = Abteilung.A_Nr 
                           AND Monteur.F_Nr = Funktion.F_Nr;""")

except MySQLdb.Error as e:
    print("MySQL Fehler:", e[0], "-", e[1])
    # print("Programm wird abgebrochen")
    # dbCursor.close()
    # dbVerbindung.close()
    # sys.exit(1)

else:
    print("SQL-Statement ausgeführt")


Konnte das SQL-Statement erfolgreich ausgeführt werden, können nun die Zeilen (Tuple, Row, Datensatz) der Tabelle (resp. des Cursors, des Result-Sets) in einer Schlaufe ausgegeben werden. Auch hier haben wir wieder zwei Varianten.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter)


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter)
    mitarbeiter = dbCursor.fetchone()


<b>Hinweis:</b> Das Result-Set kann jeweils nur einmal abgearbeitet werden. Das heisst, dass nach der Abarbeitung des Result-Sets dieses nochmals aufgebaut werden muss (sprich: das SQL-Statement muss nochmals ausgeführt werden), wenn es nochmals abgearbeitet werden soll. Dies ist normalerweise nicht notwendig, da in einer applikatorischen Verarbeitung ein Result-Set nur einmal verarbeitet werden soll/muss. Wenn in diesen Beispielen für Demonstrationszwecke das Result-Set jedoch mehrfach abgearbeitet werden soll, muss vor der Ausführung von Variante 2 das SQL-Statement (sieben Zellen oberhalb dieser Zelle) nochmals ausgeführt werden.

In beiden Varianten gibt der Cursor jede Zeile (jede Row, jeden Tupel, jeden Datensatz) als Python-Dictionary zurück. Nun können wir die Attribute über deren Key (Namen) ansprechen.

#### Variante 1: mit einem "normalem" Python-Iterator

In [ ]:
for mitarbeiter in dbCursor:
    print(mitarbeiter["M_Name"], mitarbeiter["Abteilung"], mitarbeiter["Funktion"])


#### Variante 2: mit fetchone()

In [ ]:
mitarbeiter = dbCursor.fetchone()
while mitarbeiter is not None:
    print(mitarbeiter["M_Name"], mitarbeiter["Abteilung"], mitarbeiter["Funktion"])
    mitarbeiter = dbCursor.fetchone()


Das Ansprechen der Spalten (Attribute) über deren Namen ist deutlich sprechender.

Konnte alles erfolgreich durchgeführt werden, so muss wieder sauber aufgeräumt werden. Zuerst schliessen wir den Cursor wieder.

In [ ]:
dbCursor.close()


Zum Abschluss schliessen wir auch noch die Verbindung zur Datenbank.

In [ ]:
dbVerbindung.close()


Damit haben wir auch Beispiel 4 (unser letztes Beispiel) erfolgreich abgeschlossen. 

Nun sind wir in der Lage, aus einer imperativen Programmiersprache (einer 3. Generationssprache) mit Embedded-SQL Daten aus einer relationale Datenbank zu bearbeiten (zu lesen). In weiteren IPython-Notebooks stehen zusätzliche Beispiele für die Bearbeitung (CRUD - create, read, update, delete) von Daten in relationalen Datenbanken zur Verfügung.

<b>Hinweis in eigener Sache: </b>

Welcher Cursor (Client-Side- oder Server-Side-Cursor) für das Lesen von Daten aus einer relationalen Datenbank verwendet werden soll, ist grösstenteils Geschmackssache. Nur wenn sehr grosse Datenmengen erwartet werden, drängt sich ein Server-Side-Cursor auf. 

Ob das Result-Set eines Cursors als Tupel oder als Dictionary bearbeitet wird, ist absolute Geschmackssache. Ich verwende in in der Regel die Dictionary Variante (Variante 2 in den obigen Beispielen), da dies den Source-Code von Programmen deutlich lesbarer macht. Denn wir schreiben Programme ja für unsere Nachfolger, nicht für unsere Computer.

## That's all Folks